In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
df = pd.read_csv("../input/dogecoin-historical-data/DOGE-USD.csv")

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df['Date'] =  pd.to_datetime(df['Date'])

In [ ]:
df.describe()

Plotly includes a function for creating interactive financial candlestick charts which describing open, high, low and close values for a time. The size of each box represents the spread between the open and close values, and the lines represent the spread between the low and high values. Where the close value is higher then the open value are called increasing (green). Note, to see the boxes you have to zoom-in (all the plots are interactive).

In [ ]:
df_temp = df.query('Date > 2020')

In [ ]:
fig = go.Figure(data=[go.Candlestick(
    x = df_temp['Date'],
    open = df_temp['Open'],
    close = df_temp["Close"],
    high = df_temp["High"],
    low = df_temp['Low'],
    increasing_line_color = 'green',
    decreasing_line_color = 'red'
)])

fig.update_layout(
    title='Doge Coin Value after 2020',
    yaxis_title='Doge Value (USD)',
    shapes = [dict(
    x0='2021-05-08', x1='2021-05-08', y0=0, y1=1, xref='x', yref='paper',
    line_width=2)],
    annotations=[dict(
        x='2021-05-08', y=0.05, xref='x', yref='paper',
        showarrow=False, xanchor='right', text='Elon Musk on SNL')]
)

fig.show()

In [ ]:
# daily simple return
dsr = df.copy()
dsr['DSR'] = df['Close'].pct_change(periods=1).dropna()*100 # Percentage change between the current and a prior element.

In [ ]:
fig = px.line(dsr, x='Date', y='DSR')
fig.update_layout(
    title='Daily Return',
    yaxis_title='Percent Change in Doge Value (%)',
    shapes = [dict(
        x0='2021-05-08', x1='2021-05-08', y0=0, y1=1, xref='x', yref='paper',
        line_width=2)],
    annotations=[dict(
        x='2021-05-08', y=0.05, xref='x', yref='paper',
        showarrow=False, xanchor='right', text='Elon Musk on SNL')
                ]
)

fig.show()

In [ ]:
delta = df["Close"].diff(1).dropna()
up = delta.copy()
down = delta.copy()
up[up<0] = 0
down[down>0] = 0
time_period = 14
AVG_Gain = up.rolling(window=time_period).mean()
AVG_Loss = abs(down.rolling(window=time_period).mean())
df["RS"] = AVG_Gain / AVG_Loss
df["RSI"] = 100.0 - (100.0 / (1.0 + df.RS))


The [relative strength Index](https://www.investopedia.com/terms/r/rsi.asp) of a security is a useful metric in momentum based trading algorithms. It captures the magnitude in recent prices for a stock (or in this case DOGE).  Typically, a RSI value of 70+ indicates a stock is undervalued, and an RSI of < 30 suggests a stock is overvalued (overbought).

In [ ]:
fig = px.line(df.dropna(), x="Date", y="RSI", title='Relative Strength Index of Doge')
fig.add_shape(type='line',
                x0=df.Date.min(),
                y0=70,
                x1=df.Date.max(),
                y1=70,
                line=dict(color='Red',),
                xref='x',
                yref='y'
)
fig.add_shape(type='line',
                x0=df.Date.min(),
                y0=30,
                x1=df.Date.max(),
                y1=30,
                line=dict(color='Red',),
                xref='x',
                yref='y'
)
fig.update()
fig.show()

Sometimes it's nice to visualize a moving average of the price (smoothing).

In [ ]:
SMA30 = df["Close"].rolling(window=30).mean()
df["SMA30"] = SMA30

In [ ]:
df_temp = df.query('Date > 20200701') # after July 2020
fig = px.line(df_temp, x='Date', y=['Close', 'SMA30'])
fig.update_layout(
    title='Simple Moving Average of Doge Price',
    yaxis_title='Price of Doge (USD)',
    shapes = [dict(
        x0='2021-05-08', x1='2021-05-08', y0=0, y1=1, xref='x', yref='paper',
        line_width=2)],
    annotations=[dict(
        x='2021-05-08', y=0.05, xref='x', yref='paper',
        showarrow=False, xanchor='right', text='Elon Musk on SNL')
                ]
)

fig.show()

# LSTM Forcasting

In [ ]:
import datetime
import sklearn
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
physical_devices = tf.config.list_physical_devices('GPU') 
# tf.config.experimental.set_memory_growth(physical_devices[0], True)
print("GPU power activated 🚀🚀" if len(physical_devices) > 0 else "No GPU found")

# split data in 80%/10%/10% train/validation/test sets
valid_set_size_percentage = 10 
test_set_size_percentage = 10 

In [ ]:
# leave out last 15 days for testing
dataset_train = df.iloc[:-15, :]
dataset_test = df.iloc[-15:, :]
real_stock_price = dataset_test.iloc[:, 1:2].values

In [ ]:
# just using opening and high values for modeling
sc = MinMaxScaler(feature_range = (0, 1))
training_set = dataset_train.iloc[:, 1:2].values 
training_set_scaled = sc.fit_transform(training_set)

In [ ]:
test_set = dataset_test.iloc[:, 1:2].values # just using opening and high values for modeling
test_set_scaled = sc.transform(test_set)

In [ ]:
X_train = []
y_train = []
for i in range(60, 2035):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [ ]:
def build_and_compile_model():
    regressor = Sequential([
        LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)),
        Dropout(0.2),
        LSTM(units = 50, return_sequences = True),
        Dropout(0.2),
        LSTM(units = 50, return_sequences = True),
        Dropout(0.2),
        LSTM(units = 50),
        Dropout(0.2),
        Dense(units = 1)
    ])

    regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
    return regressor

In [ ]:
log_dir = "./logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
model = build_and_compile_model()
history = model.fit(X_train, y_train, epochs = 100, batch_size = 32,
                   verbose=0, callbacks=[tensorboard_callback])
model.save('./Models/nn_model')
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
px.line(hist, x='epoch', y='loss', title='Training Loss (MSE)')

In [ ]:
dataset_total = pd.concat((dataset_train['Open'], dataset_test['Open']), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []
for i in range(60, 75):
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
predicted_stock_price = model.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

In [ ]:
df_preds = pd.DataFrame({'Date': df.loc[df.index.max()-14:, 'Date'].to_numpy(), 
                        'Real': real_stock_price.flatten(),
                        'Predicted': predicted_stock_price.flatten()
                        })

In [ ]:
fig = px.line(df_preds, x='Date', y=['Real', 'Predicted'], title='DOGE Predictions')
fig.update_layout(yaxis_title='DOGE Value (USD)').show()